# Building a Collaborative Filtering news recommender
In this notebook we preprocess and train a news recommender system based on the MIND dataset.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch

## Manual pre-processing of data
There exist a library to preprocess this data, but I find it quite hard to use it. So instead, after a quick look on the data I found it would not be too hard to do the work myself.
For the first read, it is not necessary to understand what is being done here, and I will re-introduce the dataset in the modeling section.

### behaviors.tsv

#### From documentation:  
The behaviors.tsv file contains the impression logs and users' news click histories. It has 5 columns divided by the tab symbol:
- Impression ID. The ID of an impression.  
- User ID. The anonymous ID of a user.  
- Time. The impression time with format "MM/DD/YYYY HH:MM:SS AM/PM".  
- History. The news click history (ID list of clicked news) of this user before this impression. The clicked news articles are ordered by time.  
- Impressions. List of news displayed in this impression and user's click behaviors on them (1 for click and 0 for non-click). The orders of news in a impressions have been shuffled.  

In [2]:
raw_behaviour = pd.read_csv(
    "/kaggle/input/mind-news-dataset/MINDsmall_train/behaviors.tsv", 
    sep="\t",
    names=["impressionId","userId","timestamp","click_history","impressions"])

print(f"The dataset consist of {len(raw_behaviour)} number of interactions.")
raw_behaviour.head()

The dataset consist of 156965 number of interactions.


,impressionId,userId,timestamp,click_history,impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [3]:
## Indexize users
unique_userIds = raw_behaviour['userId'].unique()
# Allocate a unique index for each user, but let the zeroth index be a UNK index:
ind2user = {idx +1: itemid for idx, itemid in enumerate(unique_userIds)}
user2ind = {itemid : idx for idx, itemid in ind2user.items()}
print(f"We have {len(user2ind)} unique users in the dataset")

# Create a new column with userIdx:
raw_behaviour['userIdx'] = raw_behaviour['userId'].map(lambda x: user2ind.get(x,0))

We have 50000 unique users in the dataset


In [4]:
raw_behaviour.head()

,impressionId,userId,timestamp,click_history,impressions,userIdx
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,1
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,2
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,3
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,4
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,5


## Load article data
We also need to get the content information of each article. We will use the news.tsv file to index the items.

In [5]:
news = pd.read_csv(
    "/kaggle/input/mind-news-dataset/MINDsmall_train/news.tsv", 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
news.head(2)
# Build index of items
ind2item = {idx +1: itemid for idx, itemid in enumerate(news['itemId'].values)}
item2ind = {itemid : idx for idx, itemid in ind2item.items()}

news.head()

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


Now we need to process the click history and impressions. We need to both indexize the strings, but also to decode impressions into clicks and non-clicks.



In [6]:
# Indexize click history field
def process_click_history(s):
    list_of_strings = str(s).split(" ")
    return [item2ind.get(l, 0) for l in list_of_strings]
        
raw_behaviour['click_history_idx'] = raw_behaviour.click_history.map(lambda s:  process_click_history(s))
raw_behaviour.head()

,impressionId,userId,timestamp,click_history,impressions,userIdx,click_history_idx
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071..."
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733..."
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ..."
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559..."
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,5,"[23643, 4853, 27686, 31189]"


In [7]:
# collect one click and one no-click from impressions:
def process_impression(s):
    list_of_strings = s.split(" ")
    itemid_rel_tuple = [l.split("-") for l in list_of_strings]
    noclicks = []
    for entry in itemid_rel_tuple:
        if entry[1] =='0':
            noclicks.append(entry[0])
        if entry[1] =='1':
            click = entry[0]
    return noclicks, click

raw_behaviour['noclicks'], raw_behaviour['click'] = zip(*raw_behaviour['impressions'].map(process_impression))
# We can then indexize these two new columns:
raw_behaviour['noclicks'] = raw_behaviour['noclicks'].map(lambda list_of_strings: [item2ind.get(l, 0) for l in list_of_strings])
raw_behaviour['click'] = raw_behaviour['click'].map(lambda x: item2ind.get(x,0))

In [8]:
raw_behaviour.head()

,impressionId,userId,timestamp,click_history,impressions,userIdx,click_history_idx,noclicks,click
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071...",[50689],33900
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733...","[37405, 41306, 34907, 35307, 44370, 37210, 439...",32187
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ...","[39528, 33356, 38720, 43459, 794, 38061, 39830...",5767
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559...","[50689, 50106, 50022]",50715
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,5,"[23643, 4853, 27686, 31189]","[2006, 33272, 39220, 37210, 45683, 50113, 3663...",31475


In [9]:
# convert timestamp value to hours since epoch
raw_behaviour['epochhrs'] = pd.to_datetime(raw_behaviour['timestamp']).values.astype(np.int64)/(1e6)/1000/3600
raw_behaviour['epochhrs'] = raw_behaviour['epochhrs'].round()

## find first publish date
#raw_behaviour[['click','epochhrs']].groupby("click").min("epochhrs").reset_index()

## Output of data preprocessing
In this preprocessing we have processed behaviour data, article data and user data.
Most importantly, we have indexized users and items in the behaviour dataframe, 
as pytorch requires integer indicies instead of strings for user and item IDs.

- Dataframe`behaviour` contains all interactions: epochhrs (a timestamp), clicks, no clicks and historical clicks per user for each interaction
- Dictionary `ind2item`: mapping the item indicies given in behaviour to the real item Id given in the dataset.
- Dictionary `ind2user`: mapping the user indicies given in behaviour to the real user Id given in the dataset.
- Dataframe `news`: content data on items. We will not use this in the first iteration.

The main component is `behaviour`, and for collaborative filtering purposes this is all we need.
However, if we want to utilize content data on the news items some preprocessing on the `news` dataframe must be used.

In [10]:
## Select the columns that we now want to use for further analysis
behaviour = raw_behaviour[['epochhrs','userIdx','click_history_idx','noclicks','click']]
behaviour.head()

,epochhrs,userIdx,click_history_idx,noclicks,click
0,437073.0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071...",[50689],33900
1,437106.0,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733...","[37405, 41306, 34907, 35307, 44370, 37210, 439...",32187
2,437143.0,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ...","[39528, 33356, 38720, 43459, 794, 38061, 39830...",5767
3,437069.0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559...","[50689, 50106, 50022]",50715
4,437104.0,5,"[23643, 4853, 27686, 31189]","[2006, 33272, 39220, 37210, 45683, 50113, 3663...",31475


# Modeling
We want to make a matrix factorization model where each user $u$ has a d-dimensional parameter vector $z_u$ and each item $i$ has a parameter vector $v_i$. This implies that we do not need the click_history_idx column when we train the model (why?).

Second, to simplify the computation of things we assume that the user only considers two items in each interaction: The item the user clicked on, and the first item in the noclicks list (what are we missing by this assumption?).

Hence, our dataset consist of a `behaviour` dataframe and a `news` dataframe with content information of the news articles. We will use a language model to interpret our article data, and for simplicity we will only use the title as the text input here.


In [11]:
behaviour.loc[:,'noclick'] = behaviour['noclicks'].map(lambda x : x[0])
behaviour.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,epochhrs,userIdx,click_history_idx,noclicks,click,noclick
0,437073.0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071...",[50689],33900,50689
1,437106.0,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733...","[37405, 41306, 34907, 35307, 44370, 37210, 439...",32187,37405
2,437143.0,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ...","[39528, 33356, 38720, 43459, 794, 38061, 39830...",5767,39528
3,437069.0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559...","[50689, 50106, 50022]",50715,50689
4,437104.0,5,"[23643, 4853, 27686, 31189]","[2006, 33272, 39220, 37210, 45683, 50113, 3663...",31475,2006


In [12]:
# Let us use the last 10pct of the data as our validation data:
test_time_th = behaviour['epochhrs'].quantile(0.9)
train = behaviour[behaviour['epochhrs']< test_time_th]
valid =  behaviour[behaviour['epochhrs']>= test_time_th]

In [13]:
class MindDataset(Dataset):
    # A fairly simple torch dataset module that can take a pandas dataframe (as above), 
    # and convert the relevant fields into a dictionary of arrays that can be used in a dataloader
    def __init__(self, df):
        # Create a dictionary of tensors out of the dataframe
        self.data = {
            'userIdx' : torch.tensor(df.userIdx.values),
            'click' : torch.tensor(df.click.values),
            'noclick' : torch.tensor(df.noclick.values)
        }
    def __len__(self):
        return len(self.data['userIdx'])
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.data.items()}

In [14]:
# Build datasets and dataloaders of train and validation dataframes:
bs = 1024
ds_train = MindDataset(df=train)
train_loader = DataLoader(ds_train, batch_size=bs, shuffle=True)
ds_valid = MindDataset(df=valid)
valid_loader = DataLoader(ds_valid, batch_size=bs, shuffle=False)

batch = next(iter(train_loader))

## Model

#### Framework
We will use pytorch-lightning to define and train our model. It is a high-level framework (similar to fastAI) but with a slightly different way of defining things. It is my personal go-to framework and is very flexible. For more information, see https://pytorch-lightning.readthedocs.io/.

#### The model
We assume that each interaction goes as follow: the user is presented with two items: the click and no-click item. After the user reviewed both items, she will choose the most relevant one. This can be modeled as a categorical distirbution with two options (yes, you could do binomial). There is a loss function in pytorch for this already, called the `F.cross_entropy` that we will use.

In [15]:
# Build a matrix factorization model
class NewsMF(pl.LightningModule):
    def __init__(self, num_users, num_items, dim = 10):
        super().__init__()
        self.dim=dim
        self.useremb = nn.Embedding(num_embeddings=num_users, embedding_dim=dim)
        self.itememb = nn.Embedding(num_embeddings=num_items, embedding_dim=dim)

    def step(self, batch, batch_idx, phase="train"):
        batch_size = batch['userIdx'].size(0)
        score_click = self.forward(batch["userIdx"], batch["click"])
        score_noclick = self.forward(batch["userIdx"], batch["noclick"])         
        scores_all = torch.concat((score_click, score_noclick), dim=1)
        loss = F.cross_entropy(input=scores_all, target=torch.zeros(batch_size, device=scores_all.device).long())
        return loss
    
    def forward(self, users, items):
        uservec =  self.useremb(users)
        itemvec = self.itememb(items)
        score = (uservec*itemvec).sum(-1).unsqueeze(-1)
        return score
               
        
    def predict_single_user(self, user_idx):
        items = torch.arange(0, len(ind2item))
        user = torch.zeros_like(items) + user_idx
        scores = self.forward(user, items)
        recommendations = [item.item() for item in torch.topk(scores, 500, dim=0)[1]]
        return recommendations
    
    def predict(self, users):
        recommendations = []
        for user in users:
            recommendation = self.predict_single_user(user)
            recommendations.append(recommendation) 
        return recommendations        
        
    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "train")
    
    def validation_step(self, batch, batch_idx):
        # for now, just do the same computation as during training
        return self.step(batch, batch_idx, "val")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
mf_model = NewsMF(num_users=len(ind2user)+1, num_items = len(ind2item)+1, dim=15)

In [16]:
trainer = pl.Trainer(max_epochs=100, accelerator="gpu")
trainer.fit(model=mf_model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## Performance

Here we get a batch of test data and calculate some performance metrics. We only have 1 true value (the item that the user actually clicked), so let's define our own version of accuracy@k as follows:

- If the item that the user clicked is in the top 10 predictions, return 1,
- Otherwise, return 0.

The mean of all predictions are a sort of accuracy@k metric. The difficulty here is of course that we only have 1 true value, otherwise we would use something like precision@k or recall@k, but those don't apply here

In [17]:
valid_batch = next(iter(valid_loader))
predictions = mf_model.predict(valid_batch["userIdx"])
true_values = [item.item() for item in valid_batch["click"]]

In [18]:
from typing import List

def accuracy_at_k(predictions: List[List], true_values: List):
    hits = 0
    for preds, true in zip(predictions, true_values):
        if true in preds:
            hits += 1
    return hits / len(true_values)

accuracy_at_k(predictions, true_values)

0.0087890625

## Playground

The cells below are a playground to inspect single recommendations

In [19]:
user_idx = 5
items = torch.arange(0, len(ind2item))
user = torch.zeros_like(items) + user_idx
recommendations = mf_model.predict_single_user(user)

In [20]:
# read news article
article_id = behaviour[behaviour.userIdx == user_idx]["click"].values[0]
news[news.itemId == ind2item[article_id]]

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
31474,N8400,autos,autosresearchguides,These are the new cars that depreciate least,Don't want to lose a ton of money on your next...,https://assets.msn.com/labs/mind/BBOA5nc.html,[],[]


In [21]:
news[news.itemId.isin([(ind2item[item + 1])  for item in recommendations])]

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
57,N10836,foodanddrink,tipsandtricks,100 Genius Tips that Will Make Your Holidays S...,From grocery shopping to cooking to using up t...,https://assets.msn.com/labs/mind/AAHYSQQ.html,"[{""Label"": ""Christmas and holiday season"", ""Ty...",[]
427,N543,music,musicnews,The 'Man in Blue'? Sheriff donates Johnny Cash...,"NASHVILLE, Tenn. (AP) A sheriff has presente...",https://assets.msn.com/labs/mind/AAJgSIa.html,[],"[{""Label"": ""Johnny Cash Museum"", ""Type"": ""N"", ..."
896,N52221,sports,football_nfl,Report: Alex Smith had 17 surgeries to repair ...,The Redskins quarterback revealed recently tha...,https://assets.msn.com/labs/mind/AAISIkc.html,"[{""Label"": ""Alex Smith"", ""Type"": ""P"", ""Wikidat...","[{""Label"": ""Washington Redskins"", ""Type"": ""O"",..."
956,N44061,finance,markets,"In manufacturing Midwest, signs of trouble ami...",A Wisconsin town shows the disparate fortunes ...,https://assets.msn.com/labs/mind/AAJwidq.html,"[{""Label"": ""Midwestern United States"", ""Type"":...","[{""Label"": ""United States"", ""Type"": ""G"", ""Wiki..."
1090,N31761,foodanddrink,recipes,45 Recipes Grandma Stole from Her Church Friends,These comforting potluck recipes are just like...,https://assets.msn.com/labs/mind/AAHB7ZO.html,[],[]
...,...,...,...,...,...,...,...,...
50897,N7012,sports,football_nfl,Browns vs. Bills Final Score: Despite Clevelan...,The losing streak is over!,https://assets.msn.com/labs/mind/BBWypjR.html,"[{""Label"": ""Cleveland Browns"", ""Type"": ""O"", ""W...",[]
50905,N10304,news,newsus,Called to serve: Stockton WWII veteran Mel Cor...,"It was the spring of 1942, and the world was f...",https://assets.msn.com/labs/mind/BBWypwM.html,[],"[{""Label"": ""Pearl Harbor"", ""Type"": ""L"", ""Wikid..."
51026,N49178,sports,football_nfl,Watch: Cardinals' Baker pulls sick move out of...,Baker topped an incredible day on the field wi...,https://assets.msn.com/labs/mind/BBWythR.html,[],[]
51071,N56774,sports,football_ncaa,Alabama-LSU delivers highest-rated regular sea...,NaN,https://assets.msn.com/labs/mind/BBWyvAp.html,"[{""Label"": ""LSU Tigers football"", ""Type"": ""F"",...",[]
